# DC Line dispatch with pandapower OPF
This is an introduction into the usage of the pandapower optimal power flow with dc lines.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf_dcline.png" width="100%">

We first create this network in pandapower:

In [1]:
import pandapower as pp
from numpy import array
net = pp.create_empty_network()

b1 = pp.create_bus(net, 380)
b2 = pp.create_bus(net, 380)
b3 = pp.create_bus(net, 380)
b4 = pp.create_bus(net, 380)
b5 = pp.create_bus(net, 380)

l1 = pp.create_line(net, b1, b2, 30, "490-AL1/64-ST1A 380.0")
l2 = pp.create_line(net, b3, b4, 20, "490-AL1/64-ST1A 380.0")
l3 = pp.create_line(net, b4, b5, 20, "490-AL1/64-ST1A 380.0")

dcl1 = pp.create_dcline(net, name="dc line", from_bus=b2, to_bus=b3, p_mw=200, loss_percent=1.0, 
                  loss_mw=0.5, vm_from_pu=1.01, vm_to_pu=1.012, max_p_mw=1000,
                  in_service=True)

eg1 = pp.create_ext_grid(net, b1, 1.02, min_p_mw=0.)
eg2 = pp.create_ext_grid(net, b5, 1.02, min_p_mw=0.)

l1 = pp.create_load(net, bus=b4, p_mw=800, controllable = False)

We now run a regular load flow to check out the DC line model:

In [2]:
pp.runpp(net)

The transmission power of the DC line is defined in the loadflow as given by the p_kw parameter, which was set to 200 MW:

In [3]:
net.res_dcline

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,200.0,152.443185,-197.5,74.491759,2.5,1.01,-0.48595,1.012,-0.725627


The losses amount to 2.5 MW, which are made up of 0.5 MW conversion loss and 200 MW * 0.01 = 2 MW transmission losses. The voltage setpoints defined at from and to bus are complied with. 

Now lets define costs for the external grids to run an OPF:

In [4]:
costeg0 = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costeg1 = pp.create_poly_cost(net, 1, 'ext_grid', cp1_eur_per_mw=8)
net.bus['max_vm_pu'] = 1.5
net.line['max_loading_percent'] = 1000

In [5]:
pp.runopp(net, delta=1e-16)

This function runs an Optimal Power Flow using the PYPOWER OPF. To make sure that the PYPOWER OPF converges, we decrease the power tolerance `delta` (the default value is `delta=1e-10`). The power tolerance `delta` is a measure of the extent to which exceeding of minimum and maximum power limits is tolerated. That is, in above case, the limits considered by the OPF for the generators are `min_p_mw - delta` and `max_p_mw + delta` as lower and upper bound respectively on the active power. 

Since we defined lower costs for Ext Grid 2, it fully services the load:

In [6]:
net.res_ext_grid

,p_mw,q_mvar
0,0.500070,-7.787527
1,805.091485,0.628364


While the DC line does not transmit any power:

In [7]:
net.res_dcline

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,0.500067,7.787487,-0.000066,-0.62707,0.500001,1.019994,-0.001448,1.013925,-1.563437


If we set the costs of the left grid to a lower value than the right grid and run the loadflow again:

In [8]:
net.poly_cost.cp1_eur_per_mw.at[costeg0] = 8
net.poly_cost.cp1_eur_per_mw.at[costeg1] = 10
pp.runopp(net, delta=1e-16)

C:\Users\mvogt\AppData\Local\Temp\ipykernel_31248\374411120.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  net.poly_cost.cp1_eur_per_mw.at[costeg0] = 8
C:\Users\mvogt\AppData\Local\Temp\ipykernel_31248\374411120.py:2: FutureWarning: Cha

We can see that the power now comes from the left ext_grid:

In [9]:
net.res_ext_grid

,p_mw,q_mvar
0,821.524623,-7.787199
1,0.000759,-21.048015


And is transmitted over the DC line:

In [10]:
net.res_dcline

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,813.573163,-26.445627,-805.022934,-21.735698,8.550229,1.011014,-2.399868,1.027504,1.522331


We can however see that the lines on the left hand side are now overloaded:

In [11]:
net.res_line.loading_percent

0    127.474837
1    124.074276
2      3.265845
Name: loading_percent, dtype: float64

If we set the maximum line loading to 100% and run the OPF again:

In [12]:
net.line["max_loading_percent"] = 100
pp.runopp(net, delta=1e-16)

We can see that the lines are no longer overloaded:

In [13]:
net.res_line.loading_percent

0    100.000007
1     97.795377
2     26.591814
Name: loading_percent, dtype: float64

Because the load is serviced from both grids:

In [14]:
net.res_ext_grid

,p_mw,q_mvar
0,644.488864,-0.680457
1,170.582480,-16.528459


And the DC line transmits only part of the power needed to service the load:

In [15]:
net.res_dcline

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,639.594579,-6.207846,-632.76691,-10.099065,6.827669,1.012431,-1.875022,1.024385,0.875621


Finally, we can also define transmission costs for the DC line:

In [16]:
costeg1 = pp.create_poly_cost(net, 0, 'dcline', cp1_eur_per_mw=3)
pp.runopp(net, delta=1e-16)

Because the sum of the costs for generating power on the left hand side (8) and transmitting it to the right side (3) is now larger than for generating on the right side (10), the OPF draws as much power from the right side as is possible without violating line loading constraints:

In [17]:
net.res_line.loading_percent

0     25.067630
1     24.770581
2    100.000029
Name: loading_percent, dtype: float64

In [18]:
net.res_dcline

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,161.063564,6.442959,-158.973825,4.897221,2.089738,1.018095,-0.467991,1.016202,-0.938724


If we relax the line loading constraint and run the OPF again:

In [19]:
net.line["max_loading_percent"] = 1000
pp.runopp(net, delta=1e-16)

The load is once again fully serviced by the grid on the right hand side:

In [20]:
net.res_ext_grid

,p_mw,q_mvar
0,0.503447,-7.787432
1,805.088098,0.628941


And the DC line is in open loop operation:

In [21]:
net.res_dcline

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,0.503444,7.787581,-0.00341,-0.626309,0.500034,1.019994,-0.001457,1.013925,-1.563423


Little consistency check:

In [22]:
net.res_ext_grid.p_mw.at[0]*8 + net.res_ext_grid.p_mw.at[1]*10 + net.res_dcline.p_from_mw.at[0]*3

np.float64(8056.418890966643)

In [23]:
net.res_cost

np.float64(8056.418890966643)